In [1]:
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
from torch.optim import Adam
import numpy as np
import matplotlib.pyplot as plt
from Neural_Nets.Classification.ClassificationDataset.Development.ClassificationDataset import *
from Neural_Nets.Classification.ElementClassifier.Development.ElementClassifier import *

In [7]:
def epoch(net, train_loader, loss_func, optimizer, batch_size, seq_len):
    epoch_losses = np.zeros([len(train_loader), ])
    
    for i, d in enumerate(train_loader):
        # Scale inputs and get predictions
        inp = d[:, :, :-1].squeeze(1)/1000
        predictions = net(inp.float())
        
        # Each batch consists of measurement batches, where seq_len measurements are put into one batch. In such a 
        # measurement batch, every measurement has the same label as it needs to be from the same element an phase. 
        # This leads to a target array where the target class is contained for each measurement in the measurement 
        # batch. With this, CrossEntropyLoss would not work as the predictions are made for the whole measurement 
        # batch and CEL therefore expects only on class label per measurement batch. Therefore, only the first 
        # element of the last dimension of d is considered as target (all the entries in the last dimension are the 
        # same anyways so it could be any entry)
        targets = d[:, :, -1][:, 0].long()
        
        # Calculate the loss
        loss = loss_func(predictions, targets)
        epoch_losses[i] = loss
        
        # Backward step
        net.zero_grad()
        loss.backward()
        optimizer.step()
        
    print(epoch_losses.mean())
    
    return epoch_losses.mean()

In [14]:
# Training and testing routines

def train(net, train_loader, batch_size, seq_len, save=True):
    # Hyperparameters
    nr_epochs = 1000
    lr = 0.0025
    
    loss_func = nn.CrossEntropyLoss()
    optimizer = Adam(net.parameters(), lr=lr)
    
    losses = np.zeros([nr_epochs, ])
    
    for i in range(nr_epochs):
        losses[i] = epoch(net, train_loader, loss_func, optimizer, batch_size, seq_len)
        
def test(net, test_loader): 
    correct = 0
    incorrect = 0

    for d in test_loader:
        inp = d[:, :, :-1].squeeze(1)/1000
        prediction = net(inp.float()).argmax()
        target = d[:, :, -1][:, 0].long()
        if prediction == target:
            correct += 1
        else:
            incorrect += 1

    accuracy = correct/(correct + incorrect)
    print('Test accuracy: ', accuracy)
    
    return accuracy

In [4]:
# Hyperparameters
seq_len = 10
measurement = 'C'

dc = DatasetCreator(r"C:\Users\danie\Documents\Montanuni\Masterarbeit\4_Daten\Elements.xlsx", elements=None, 
                   splits=(0.8, 0.2), validation=False, seq_len=seq_len, measurement=measurement, stable_only=True)
train_dataset, test_dataset, val_dataset = dc.get_datasets()

In [19]:
# Hyperparameters
batch_size = 32

# Create the DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)
if val_dataset:
    val_loader = DataLoader(val_dataset)
    
# Create the network
t = False
net = ElementClassifier(train)

In [20]:
if t:
    train(net, train_loader, batch_size, seq_len)
else: 
    test(net, test_loader)

Test accuracy:  0.016218354430379747
tensor(17) tensor([17])
